In [56]:
import sys, os
from pymongo import MongoClient

import pandas as pd

import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *

# https://stackoverflow.com/questions/41323423/plotly-inside-jupyter-notebook-python
init_notebook_mode(connected=True) # initiate notebook for offline plot

In [57]:
sys.path.append("..")
import config # ../config.py
import common # ../common.py

# run on dellxps
config.mongodb_ip = "192.168.1.114"

client = MongoClient(host=config.mongodb_ip, port=config.mongodb_port)
db = client['steam']

In [58]:
dates_df = pd.DataFrame(list(db['apps'].aggregate
([
        {'$match': {"$or":[
            {'type': "game"},
            {'type': "dlc"}
        ]}},
    {"$project": {"_id":False, "appid":1, "release_date":"$release_date.date"}}
]))).dropna()

dates_df

,appid,release_date
0,10,2000-11-01 00:00:00
1,20,1999-04-01 00:00:00
2,30,2003-05-01 00:00:00
3,40,2001-06-01 00:00:00
4,50,1999-11-01 00:00:00
...,...,...
66110,1189080,2020-01-31 00:00:00
66111,1197440,2020-02-10 00:00:00
66112,620410,TBA
66113,1228410,2020-02-26 00:00:00


In [59]:
dates_df.release_date = pd.to_datetime(dates_df.release_date, format='%Y-%m-%d', errors='coerce')

dates_df = dates_df[pd.notnull(dates_df['release_date'])]

dates_df

,appid,release_date
0,10,2000-11-01
1,20,1999-04-01
2,30,2003-05-01
3,40,2001-06-01
4,50,1999-11-01
...,...,...
66109,976570,2020-01-29
66110,1189080,2020-01-31
66111,1197440,2020-02-10
66113,1228410,2020-02-26


In [60]:
price_hist_df = pd.DataFrame(list(db['pricehistory'].find({"discount_percent": {"$ne":0}}, {"appid":1,"discount_percent":1,"date":1,"_id":False})))

price_hist_df

,appid,date,discount_percent
0,270270,2018-09-04 03:05:16.176,60
1,371200,2018-09-04 03:05:20.772,67
2,283940,2018-09-04 03:05:20.774,60
3,717370,2018-09-04 03:05:25.393,51
4,933890,2018-09-04 03:05:32.206,40
...,...,...,...
1255574,262142,2020-01-19 02:53:12.301,50
1255575,543100,2020-01-19 02:53:12.404,20
1255576,622530,2020-01-19 02:53:19.359,46
1255577,541450,2020-01-19 02:53:19.422,75


In [61]:
filtered_df = price_hist_df.query('discount_percent>=50')
# sanity check:
#filtered_df.query("appid==552520").sort_values(by='date', ascending=True)

# this will find the earliest occurence of a discount per appid
filtered_df = filtered_df.loc[filtered_df.groupby(filtered_df.appid, as_index=False).date.idxmin()]

# https://isthereanydeal.com/game/farcryv/history/?shop%5B%5D=steam&generate=Select+Stores
# appid: 552520 is Far Cry 5
# this test should return one entry, the earliest timepoint at which the game met or
# exceeded that discount percentage
#filtered_df.query("appid==552520")

filtered_df

,appid,date,discount_percent
80446,10,2018-11-22 00:51:24.561,90
90349,20,2018-11-22 01:18:25.141,80
84229,30,2018-11-22 01:01:36.698,80
85941,40,2018-11-22 01:06:16.253,80
78642,50,2018-11-22 00:46:22.991,80
...,...,...,...
983096,1210990,2019-12-20 06:11:35.872,50
1237412,1211490,2020-01-07 16:29:23.113,51
1255239,1212860,2020-01-18 21:10:31.943,50
983100,1213480,2019-12-20 06:12:24.770,80


In [62]:
model_df = pd.merge(dates_df,
                 filtered_df,
                 on='appid', 
                 how='left').dropna()
# drop NAs removes the cases where we have no price history information

model_df

,appid,release_date,date,discount_percent
0,10,2000-11-01,2018-11-22 00:51:24.561,90.0
1,20,1999-04-01,2018-11-22 01:18:25.141,80.0
2,30,2003-05-01,2018-11-22 01:01:36.698,80.0
3,40,2001-06-01,2018-11-22 01:06:16.253,80.0
4,50,1999-11-01,2018-11-22 00:46:22.991,80.0
...,...,...,...,...
61600,1211490,2019-12-27,2020-01-07 16:29:23.113,51.0
61608,1209590,2019-12-18,2020-01-18 22:27:24.425,80.0
61853,1210990,2019-12-19,2019-12-20 06:11:35.872,50.0
61875,1213480,2019-12-19,2019-12-20 06:12:24.770,80.0


In [63]:
model_df['length_in_days_since_release'] = (model_df['date'] - model_df['release_date']).dt.days

model_df = model_df.sort_values(by='length_in_days_since_release', ascending=True)

model_df

,appid,release_date,date,discount_percent,length_in_days_since_release
22050,557580,2019-12-23,2018-09-27 02:06:28.267,90.0,-452
6373,298280,2019-12-24,2018-11-01 00:57:12.592,70.0,-418
32849,744040,2019-12-12,2018-11-01 00:25:41.299,50.0,-406
29015,676760,2019-10-13,2018-09-14 01:32:23.786,83.0,-394
7973,328080,2019-10-14,2018-09-17 23:49:34.016,50.0,-392
...,...,...,...,...,...
4,50,1999-11-01,2018-11-22 00:46:22.991,80.0,6961
1,20,1999-04-01,2018-11-22 01:18:25.141,80.0,7175
6,70,1998-11-08,2018-11-22 01:02:40.211,80.0,7319
3424,232770,1997-11-14,2018-09-18 00:04:38.704,75.0,7613


In [64]:
grped_df = model_df.groupby(['length_in_days_since_release']).size().reset_index(name='counts')
grped_df

,length_in_days_since_release,counts
0,-452,1
1,-418,1
2,-406,1
3,-394,1
4,-392,1
...,...,...
3166,6961,1
3167,7175,1
3168,7319,1
3169,7613,1


In [65]:
# https://plot.ly/python/bar-charts/

data = [Bar(
        x=grped_df['length_in_days_since_release'],
        y=grped_df['counts']
    )]

layout = Layout(
        title='Amount in days since the release until each game hits discount percentage mark',
        xaxis=dict(
            title='Amount in days since release'
        ),
        yaxis=dict(
            title='Count'
        )
    )

fig = Figure(data=data, layout=layout)

iplot(fig)

In [66]:
opencriticavg_df = pd.DataFrame(list(
    db['opencritic'].aggregate([
        {'$match': {"$and":[
            {'steamId': {'$exists':True}},
            {'steamId': {'$ne':None}},
            {'steamId': {'$ne': ''}},
            {'steamId': {'$ne':'null'}},
            {'steamId': {'$ne':'NaN'}}
        ]}},
        {"$project" : {"appid":"$steamId", "avgscore":{"$avg": "$Reviews.score"}}},
        {'$match':{'avgscore': {'$ne': None}}},
        {"$project": {"_id": 0}}
    ])
)).sort_values(by='appid', ascending=False)

opencriticavg_df = opencriticavg_df.astype({"appid": int})

opencriticavg_df

,appid,avgscore
4642,998990,83.750000
4641,997060,58.800000
4640,996470,81.444444
4639,995860,70.000000
4638,995430,37.555556
...,...,...
4,1001570,50.000000
3,1001140,75.400000
2,1001060,75.400000
1,1000640,57.500000


In [67]:
model_df = pd.merge(model_df,
                 opencriticavg_df,
                 on='appid', 
                 how='left').dropna()

model_df

,appid,release_date,date,discount_percent,length_in_days_since_release,avgscore
9,487120,2019-11-01,2018-11-01 00:11:46.609,50.0,-365,66.000000
21,605100,2019-08-28,2018-11-01 00:58:13.274,60.0,-300,30.000000
24,526160,2019-10-03,2018-12-21 19:44:43.556,50.0,-286,75.000000
25,602490,2019-07-10,2018-09-28 23:02:53.090,50.0,-285,40.000000
32,489520,2019-05-24,2018-09-11 01:03:50.567,70.0,-255,80.000000
...,...,...,...,...,...,...
30513,40340,2009-10-29,2018-10-26 00:06:36.038,75.0,3284,63.333333
30525,21690,2009-09-15,2018-09-20 02:52:02.122,75.0,3292,69.000000
30571,6030,2009-09-16,2018-11-08 01:26:25.146,65.0,3340,62.800000
30649,35700,2009-07-02,2018-11-01 00:53:14.619,85.0,3409,80.700000


In [68]:
recommendations_df = pd.DataFrame(list(
    db['apps'].aggregate([
        {"$project": {"_id": False, "appid": 1, "steam_recommendations": "$recommendations.total"}}
    ])
)).fillna(0)

recommendations_df

,appid,steam_recommendations
0,10,80232.0
1,20,3235.0
2,30,2379.0
3,40,1114.0
4,50,5476.0
...,...,...
91576,1189080,0.0
91577,1197440,0.0
91578,620410,0.0
91579,1228410,0.0


In [69]:
model_df = pd.merge(model_df,
                 recommendations_df,
                 on='appid', 
                 how='left').dropna()

model_df

,appid,release_date,date,discount_percent,length_in_days_since_release,avgscore,steam_recommendations
0,487120,2019-11-01,2018-11-01 00:11:46.609,50.0,-365,66.000000,3977.0
1,605100,2019-08-28,2018-11-01 00:58:13.274,60.0,-300,30.000000,0.0
2,526160,2019-10-03,2018-12-21 19:44:43.556,50.0,-286,75.000000,4332.0
3,602490,2019-07-10,2018-09-28 23:02:53.090,50.0,-285,40.000000,0.0
4,489520,2019-05-24,2018-09-11 01:03:50.567,70.0,-255,80.000000,1315.0
...,...,...,...,...,...,...,...
3350,40340,2009-10-29,2018-10-26 00:06:36.038,75.0,3284,63.333333,0.0
3351,21690,2009-09-15,2018-09-20 02:52:02.122,75.0,3292,69.000000,9304.0
3352,6030,2009-09-16,2018-11-08 01:26:25.146,65.0,3340,62.800000,2178.0
3353,35700,2009-07-02,2018-11-01 00:53:14.619,85.0,3409,80.700000,8268.0


In [70]:
x = model_df.iloc[:, 5:6].values # predictive feature engineering inputs
y = model_df.iloc[:, 4].values # what we're trying to predict

X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [71]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [72]:
regressor = RandomForestRegressor(n_estimators=600, random_state=0)
regressor.fit(X_train, Y_train)
Y_pred = regressor.predict(X_test)

In [73]:
# https://medium.com/acing-ai/how-to-evaluate-regression-models-d183b4f5853d
print('Mean Absolute Error:', metrics.mean_absolute_error(Y_test, Y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(Y_test, Y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(Y_test, Y_pred)))

Mean Absolute Error: 421.36596950200015
Mean Squared Error: 317752.1832697364
Root Mean Squared Error: 563.695115527655
